### Imports & Definitions

In [2]:
import sys
sys.path.append("../../utils")
from definitions import *
from path_helpers import get_dataset_path, get_metric_dir_path

### Process results

In [3]:
with open(ARTIFACTS_DIR_PATH.joinpath("datasets/diplomas_asessors_questions/mcs_df_human_filled_processed.json"), "r") as f:
    dataset = json.load(f)

In [6]:
rows = []
pred_by_model = dict()
for baseline_model in HUGGINFACE_BASELINE_MODELS:
    splits = ["abstract", "empty", "diploma"]
    if baseline_model == LLAMA_2_7B:
        print("ok")
        splits.extend(["learnt", "learnt_8k"])
        print(splits)
    for split in splits:
        metric_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/{split}/{baseline_model}/") # get_metric_dir_path(baseline_model, subset, name, split)
        with open(metric_path.joinpath("answers.jsonl"), "r") as f:
            answers = json.load(f)
        pred = [x["answer"] for x in answers]
        pred_by_model[f"{split}_{baseline_model}"] = pred
        true = [x["outputs"] for x in dataset]
        rows.append({
            "model": baseline_model,
            "subset": "asessors_questions",
            "split": f"{split}_8000" if split == "diploma" else split,
            "accuracy_score": accuracy_score(true, pred),
        })
df = pd.DataFrame(rows)
df.to_csv(METRICS_DIR_PATH/"asessors.csv")
df

ok
['abstract', 'empty', 'diploma', 'learnt', 'learnt_8k']


,model,subset,split,accuracy_score
0,mistral-7b,asessors_questions,abstract,0.285714
1,mistral-7b,asessors_questions,empty,0.185714
2,mistral-7b,asessors_questions,diploma_8000,0.257143
3,llama-2-7b,asessors_questions,abstract,0.285714
4,llama-2-7b,asessors_questions,empty,0.228571
5,llama-2-7b,asessors_questions,diploma_8000,0.200000
6,llama-2-7b,asessors_questions,learnt,0.228571
7,llama-2-7b,asessors_questions,learnt_8k,0.271429


In [26]:
for i in range(6):
    for j in range(i):
        a = np.array(list(pred_by_model.values())[i])
        b = np.array(list(pred_by_model.values())[j])
        print(i, j, np.where(a != b))

1 0 (array([], dtype=int64),)
2 0 (array([], dtype=int64),)
2 1 (array([], dtype=int64),)
3 0 (array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
       63, 65, 66, 69]),)
3 1 (array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
       63, 65, 66, 69]),)
3 2 (array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
       63, 65, 66, 69]),)
4 0 (array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
       63, 65, 66, 69]),)
4 1 (array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
       63, 65, 66, 69]),)
4 2 (array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
       63, 65, 66, 69]),)
4 3 (array([], dtype=int64),)
5 0 (array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
       63, 65, 66, 69]),)
5 1 (array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
       63, 65, 66, 69]),)
5 2 (array([ 2,  6, 16, 24, 25, 

In [15]:
a, b = pred_by_model.values()
a = np.array(a) 
b = np.array(b)
np.where(a != b)

(array([ 2,  6, 16, 24, 25, 26, 35, 41, 42, 49, 50, 52, 53, 54, 56, 58, 62,
        63, 65, 66, 69]),)

In [12]:
rows = []
for baseline_model in HUGGINFACE_BASELINE_MODELS.union([VIKHR_7B, SAIGA_MISTRAL_7B_LORA]):
    for name, dataset_meta in HUGGINGFACE_NAME_TO_DATASET.items():
        for subset, split in zip(dataset_meta["subsets"], dataset_meta["splits"]):
            dataset_path = get_dataset_path(subset, name, split)
            dataset = load_from_disk(dataset_path)
            metric_path = get_metric_dir_path(baseline_model, subset, name, split)
            with open(metric_path.joinpath("answers.jsonl"), "r") as f:
                answers = json.load(f)
            pred = [x["answer"] for x in answers]
            true = [x["outputs"] for x in dataset]
            rows.append({
                "model": baseline_model,
                "subset": subset,
                "split": split,
                "accuracy_score": accuracy_score(true, pred),
            })
df = pd.DataFrame(rows)
df.to_csv(METRICS_DIR_PATH/"ru_metrics.csv")
df

,model,subset,split,accuracy_score
0,saiga_mistral_7b_lora,rummlu,public_test,0.447424
1,saiga_mistral_7b_lora,ruopenbookqa,train,0.602652
2,vikhr-7b,rummlu,public_test,0.309279
3,vikhr-7b,ruopenbookqa,train,0.336185
4,mistral-7b,rummlu,public_test,0.423203
5,mistral-7b,ruopenbookqa,train,0.525663
6,llama-2-7b,rummlu,public_test,0.301106
7,llama-2-7b,ruopenbookqa,train,0.262618


In [23]:
df.loc[df["subset"] == "rummlu"].sort_values(by="accuracy_score", ascending=False)

,model,subset,split,accuracy_score
0,saiga_mistral_7b_lora,rummlu,public_test,0.447424
4,mistral-7b,rummlu,public_test,0.423203
2,vikhr-7b,rummlu,public_test,0.309279
6,llama-2-7b,rummlu,public_test,0.301106


In [24]:
df.loc[df["subset"] == "ruopenbookqa"].sort_values(by="accuracy_score", ascending=False)

,model,subset,split,accuracy_score
1,saiga_mistral_7b_lora,ruopenbookqa,train,0.602652
5,mistral-7b,ruopenbookqa,train,0.525663
3,vikhr-7b,ruopenbookqa,train,0.336185
7,llama-2-7b,ruopenbookqa,train,0.262618
